In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np

import random
#import memory_profiler
import altair as alt
from tqdm import tqdm
import datetime

USE_ALL_STOCK_IDS = True
TEST_SIZE = 0.25

import os
if os.environ.get('USER') == 'ian':
    import ipython_memory_usage
    %ipython_memory_usage_start
else:
    assert USE_ALL_STOCK_IDS, "Must use all stocks if on Kaggle platform"
    
    # kaggle notes:
    # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
    # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 133.08 MiB


In [2]:
import os
import pandas as pd
from tqdm import tqdm

if os.environ.get("USER") == "ian":
    ROOT = "/home/ian/data/kaggle/optiver_volatility/"
else:
    ROOT = "/kaggle/input/optiver-realized-volatility-prediction"
print(f"Utility says ROOT is {ROOT}")

TRAIN_CSV = os.path.join(ROOT, "train.csv")
TEST_CSV = os.path.join(ROOT, "test.csv")


def get_training_stock_ids(parquet_folder="book_train.parquet"):
    return [
        int(s.split("=")[1]) for s in os.listdir(os.path.join(ROOT, parquet_folder))
    ]


def get_data(verbose=True, stock_ids=None):
    """stock_ids can be None (uses 0..9 by default) or 'all' for all stocks or a specified list of numeric ids"""
    items_in_folder = os.listdir(os.path.join(ROOT, "book_train.parquet"))
    if verbose:
        print(
            f"There are {len(items_in_folder)} items in the folder and they look like {items_in_folder[:5]}"
        )

    # if stock_ids == 'all':
    #    1/0
    #    #stock_ids = range(127) # not a contiguous range!
    #    #stock_ids = get_training_stock_ids()
    # if not stock_ids:
    #    stock_ids = range(10) # [0..126] files in the total set
    if True:
        stock_filenames = []
        #for stock_id in tqdm(stock_ids):
        #    assert isinstance(stock_id, int)
        #    stock_filenames.append(os.path.join(ROOT, f"book_train.parquet/stock_id={stock_id}"))
        df_book_train = pd.read_parquet(os.path.join(ROOT, f"book_train.parquet"))

    if False:
        df_book_trains = []
        for stock_id in tqdm(stock_ids):
            assert isinstance(stock_id, int)
            df_book_train_stock_X = pd.read_parquet(
                os.path.join(ROOT, f"book_train.parquet/stock_id={stock_id}")
            )
            df_book_train_stock_X["stock_id"] = stock_id
            df_book_trains.append(df_book_train_stock_X)
        #df_book_train = pd.concat(df_book_trains)
        df_book_train = pd.concat(df_book_trains, copy=False)

    if False:
        df_book_train = None
        for stock_id in tqdm(stock_ids):
            assert isinstance(stock_id, int)
            df_book_train_stock_X = pd.read_parquet(
                os.path.join(ROOT, f"book_train.parquet/stock_id={stock_id}")
            )
            df_book_train_stock_X["stock_id"] = stock_id
            if df_book_train is None:
                df_book_train = df_book_train_stock_X
            else:
                df_book_train = pd.concat((df_book_train, df_book_train_stock_X),)
                
                
    if False:
        # doesn't even finish...
        from dask.distributed import Client
        if 'client' not in dir():
            # useful for Pandas - no threads (Pandas not GIL-friendly), many processes
            # and enough memory to not max out my laptop
            client = Client(processes=True, n_workers=2, threads_per_worker=1, memory_limit='10GB')
        print(client) # show client details                

        import dask.dataframe as dd
        ddf = dd.read_parquet(path=os.path.join(ROOT, 'book_train.parquet'))
        df_book_train = ddf.compute()    

    if verbose:
        print(
            f"Loaded {df_book_train.shape[0]:,} rows for book_train on {len(stock_ids)} stock_ids"
        )

    df_train_all = pd.read_csv(TRAIN_CSV)
    training_rows_was = df_train_all.shape[0]
    df_train_all = df_train_all.query("stock_id in @stock_ids")
    training_rows_is = df_train_all.shape[0]
    if training_rows_was != training_rows_is:
        print(f"**** Had {training_rows_was:,} rows, now we have {training_rows_is:,}")
    else:
        print(f"Kept all training rows during get_data")

    return df_train_all, df_book_train


def make_unique_time_ids(all_time_ids, test_size=0.33):
    first_n_upper_bound = int(len(all_time_ids) * (1 - test_size))
    print(f"Taking {first_n_upper_bound} for train")
    return set(all_time_ids[:first_n_upper_bound]), set(
        all_time_ids[first_n_upper_bound:]
    )

Utility says ROOT is /home/ian/data/kaggle/optiver_volatility/
In [2] used 0.5078 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 133.59 MiB


## Load train set

In [3]:
stock_ids = get_training_stock_ids('book_train.parquet') # all stocks by default
if not USE_ALL_STOCK_IDS:
    # choose a random subset
    print("Using a subset")
    random.shuffle(stock_ids)
    stock_ids = stock_ids[:30]

In [3] used 0.1953 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 133.78 MiB


In [4]:
if 'df_train_all' not in dir():
    # only reload if necessary
    print(f"Loading {len(stock_ids)} stocks")
    df_train_all, df_book_train = get_data(stock_ids=stock_ids)
    
    df_train_all = df_train_all.set_index(['stock_id', 'time_id'])
    #df_train_all

Loading 112 stocks
There are 112 items in the folder and they look like ['stock_id=59', 'stock_id=58', 'stock_id=23', 'stock_id=17', 'stock_id=109']
Loaded 167,253,289 rows for book_train on 112 stock_ids
Kept all training rows during get_data
In [4] used 12578.6953 MiB RAM in 7.92s, peaked 630.61 MiB above current, total RAM usage 12712.48 MiB


In [5]:
all_time_ids = df_book_train.time_id.unique()
np.random.shuffle(all_time_ids) # shuffle the time_ids
print(f"We have {len(all_time_ids):,} time ids")
time_ids_train, time_ids_test = make_unique_time_ids(all_time_ids, test_size=TEST_SIZE)
assert len(time_ids_train) + len(time_ids_test) == len(all_time_ids)
assert len(time_ids_train.intersection(time_ids_test)) == 0, "Expecting no overlap between train and test time ids"
print(f"Example time ids for training, min first: {sorted(list(time_ids_train))[:5]}")

We have 3,830 time ids
Taking 2872 for train
Example time ids for training, min first: [11, 16, 31, 62, 72]
In [5] used -1881.3438 MiB RAM in 1.21s, peaked 1881.34 MiB above current, total RAM usage 10831.13 MiB


In [6]:
if USE_ALL_STOCK_IDS:
    assert len(stock_ids) == 112
    if not (df_train_all.groupby('time_id').count().max()[0] == 112 and \
      df_train_all.groupby('time_id').count().min()[0] == 111):
        display(df_train_all.groupby('time_id').count().agg(['max', 'min']))
        raise Warning('Odd! Expecting 111-112')

In [6] used -672.5508 MiB RAM in 0.14s, peaked 278.70 MiB above current, total RAM usage 10158.58 MiB


# Overview

In [7]:
print(df_train_all.shape)
df_train_all.head()

(428932, 1)


target
stock_id time_id          
0        5        0.004136
         11       0.001445
         16       0.002168
         31       0.002195
         62       0.001747

In [7] used -600.1094 MiB RAM in 0.13s, peaked 600.11 MiB above current, total RAM usage 9558.47 MiB


# Features

In [8]:
df_book_train.shape

(167253289, 11)

In [8] used -603.8633 MiB RAM in 0.12s, peaked 342.36 MiB above current, total RAM usage 8954.61 MiB


In [9]:
df_book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [9] used -1403.2617 MiB RAM in 0.18s, peaked 694.06 MiB above current, total RAM usage 7551.35 MiB


In [10]:
# make feature columns
cols = ['bid_price1', 'ask_price1']
def make_features_var1(df_book):
    features_var1 = df_book.groupby(['stock_id', 'time_id'])[cols].var()
    features_var1_col_names = [f"{col}_var" for col in cols]
    features_var1.columns = features_var1_col_names
    #features_var1
    return features_var1
    
features_var1 = make_features_var1(df_book_train)
features_var1

bid_price1_var  ask_price1_var
stock_id time_id                                
0        5          3.557702e-07    3.609267e-07
         11         8.048454e-08    4.782809e-08
         16         5.057495e-07    6.204843e-07
         31         4.746649e-07    3.223060e-07
         62         4.740689e-08    3.653909e-08
...                          ...             ...
99       32751      1.999395e-07    1.984993e-07
         32753      3.945513e-07    4.257106e-07
         32758      4.122112e-07    4.117799e-07
         32763      9.529251e-08    9.768070e-08
         32767      4.200461e-08    4.372325e-08

[428960 rows x 2 columns]

In [10] used -1370.5664 MiB RAM in 21.60s, peaked 6490.73 MiB above current, total RAM usage 6180.78 MiB


# Merge

In [11]:
train_merged = pd.merge(df_train_all, features_var1, left_index=True, right_index=True)
train_merged

target  bid_price1_var  ask_price1_var
stock_id time_id                                          
0        5        0.004136    3.557702e-07    3.609267e-07
         11       0.001445    8.048454e-08    4.782809e-08
         16       0.002168    5.057495e-07    6.204843e-07
         31       0.002195    4.746649e-07    3.223060e-07
         62       0.001747    4.740689e-08    3.653909e-08
...                    ...             ...             ...
126      32751    0.003461    2.528495e-07    2.074568e-07
         32753    0.003113    1.594177e-06    1.699587e-06
         32758    0.004070    2.470041e-07    1.960531e-07
         32763    0.003357    1.761607e-07    1.756424e-07
         32767    0.002090    1.205763e-07    1.261233e-07

[103409 rows x 3 columns]

In [11] used 55.4844 MiB RAM in 0.66s, peaked 0.00 MiB above current, total RAM usage 6236.27 MiB


In [12]:
features = train_merged.drop(columns='target').columns

In [12] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 6236.27 MiB


In [13]:
chs = []
for feature in features:
    print(f"Making {feature}")
    df_sample = train_merged.reset_index().sample(5_000)
    chs.append(alt.Chart(df_sample).mark_point().encode(x=feature, y='target'))
    
# https://altair-viz.github.io/user_guide/compound_charts.html
alt.vconcat(*chs)

Making bid_price1_var
Making ask_price1_var


alt.VConcatChart(...)

In [13] used 3.7109 MiB RAM in 0.72s, peaked 0.50 MiB above current, total RAM usage 6239.98 MiB


In [14]:
train_merged.reset_index()

,stock_id,time_id,target,bid_price1_var,ask_price1_var
0,0,5,0.004136,3.557702e-07,3.609267e-07
1,0,11,0.001445,8.048454e-08,4.782809e-08
2,0,16,0.002168,5.057495e-07,6.204843e-07
3,0,31,0.002195,4.746649e-07,3.223060e-07
4,0,62,0.001747,4.740689e-08,3.653909e-08
...,...,...,...,...,...
103404,126,32751,0.003461,2.528495e-07,2.074568e-07
103405,126,32753,0.003113,1.594177e-06,1.699587e-06
103406,126,32758,0.004070,2.470041e-07,1.960531e-07
103407,126,32763,0.003357,1.761607e-07,1.756424e-07


In [14] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 6239.98 MiB


In [15]:
#time_ids_train, time_ids_test
def train_test_split(df, target_col, time_ids_train, time_ids_test):
    X_train = df.query('time_id in @time_ids_train').drop(columns=[target_col, 'time_id'])
    X_test = df.query('time_id in @time_ids_test').drop(columns=[target_col, 'time_id'])
    y_train = df.query('time_id in @time_ids_train')[target_col]
    y_test = df.query('time_id in @time_ids_test')[target_col]
    return X_train, X_test, y_train, y_test

#feature_cols = ['bid_price1_var', 'ask_price1_var', 'stock_id', 'time_id']
#X_train, X_test, y_train, y_test = train_test_split(train_merged.reset_index()[feature_cols+['target']], 'target', time_ids_train, time_ids_test)

feature_cols = ['bid_price1_var', 'ask_price1_var', 'stock_id']
X_train, X_test, y_train, y_test = train_test_split(train_merged.reset_index()[feature_cols+['time_id', 'target']], 'target', time_ids_train, time_ids_test)

In [15] used 0.0000 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 6239.98 MiB


In [16]:
X_train.head(3)

,bid_price1_var,ask_price1_var,stock_id
1,8.048454e-08,4.782809e-08,0
2,5.057495e-07,6.204843e-07,0
3,4.746649e-07,3.223060e-07,0


In [16] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 6239.98 MiB


In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((77543, 3), (25866, 3), (77543,), (25866,))

In [17] used 0.0195 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 6240.00 MiB


# ML on a train/test split

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [18] used 30.8633 MiB RAM in 0.93s, peaked 0.00 MiB above current, total RAM usage 6270.86 MiB


In [19]:
#est = LinearRegression()
est = RandomForestRegressor(n_estimators=10) # default n_estimators==100

In [19] used 0.0508 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 6270.91 MiB


In [20]:
est.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10)

In [20] used 8.2266 MiB RAM in 1.90s, peaked 0.00 MiB above current, total RAM usage 6279.14 MiB


In [21]:
print(f"{df_train_all.reset_index().stock_id.unique().shape[0]} unique stock ids, test set is {TEST_SIZE*100:0.1f}%")
print(f"Features:", feature_cols)
score = est.score(X_test, y_test) # r^2
print(f"r^2 score {score:0.3f}")

112 unique stock ids, test set is 25.0%
Features: ['bid_price1_var', 'ask_price1_var', 'stock_id']
r^2 score 0.436
In [21] used 13.1836 MiB RAM in 0.15s, peaked 0.00 MiB above current, total RAM usage 6292.32 MiB


In [22]:
if 'feature_importances_' in dir(est):
    feature_col = 'feature_importances_'
if 'coef_' in dir(est):
    feature_col = 'coef_'
df_features = pd.DataFrame(zip(getattr(est, feature_col), feature_cols), columns=['importance', 'feature']).set_index('importance')
df_features.sort_index(ascending=False)

,feature
importance,
0.422491,ask_price1_var
0.367111,bid_price1_var
0.210398,stock_id


In [22] used 0.0000 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 6292.32 MiB


# Load test set

In [23]:
%xdel df_book_train

In [23] used 0.2578 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 6292.58 MiB


# Make predictions

In [24]:
len(stock_ids) # expecting 112

112

In [24] used 0.0195 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 6292.60 MiB


In [25]:
# end of notebook

In [25] used 0.1250 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 6292.72 MiB


In [26]:
# make a subset of stocks for local testing
if False:
    stock_ids_for_small_test_set = range(120, 127)
    dfx = df_train_all.query('stock_id in @stock_ids_for_small_test_set')
    ser_row_id = dfx.reset_index()[['stock_id', 'time_id']].apply(lambda x: f"{x[0]}-{x[1]}", axis=1)
    dfx2 = dfx.reset_index()
    dfx2['row_id'] = ser_row_id
    dfx2.to_csv('test_local.csv', index=False)

In [26] used -0.0117 MiB RAM in 0.11s, peaked 0.01 MiB above current, total RAM usage 6292.71 MiB


In [49]:
df_test_all.shape[0]

1

In [49] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 6341.98 MiB


In [50]:
if False: # True if debugging
    # book train as a substitute
    df_test_all = pd.read_csv(os.path.join(ROOT, 'test_local.csv'))
    df_test_all = df_test_all.rename(columns={'target': 'train_target'})
    TEST_FOLDER = 'book_test_local.parquet'
else:
    df_test_all = pd.read_csv(TEST_CSV)
    if df_test_all.shape[0] == 3: # kaggle test data
        df_test_all = df_test_all[:1] # cut out 2 rows so predictions work
    TEST_FOLDER = 'book_test.parquet'
print(TEST_CSV)
print(TEST_FOLDER)
df_test_all = df_test_all.set_index(['stock_id', 'time_id'])


/home/ian/data/kaggle/optiver_volatility/test.csv
book_test.parquet
In [50] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 6341.98 MiB


In [51]:
test_set_predictions = []
stock_ids_test = get_training_stock_ids(TEST_FOLDER) # all stocks by default

# NOTE that the provided test data has a mismatch on the number of rows, the parquet
# file for stock_id 0 only has 1 time_id (4) and 3 seconds_in_bucket (0, 1, 5)
# whilst the test.csv has stock_id 0 with 3 time_ids, so this can't work. Frustrating.

df_test_predictions = pd.DataFrame() # prediction set to build up
for stock_id in tqdm(stock_ids_test):
    parquet_filename = os.path.join(ROOT, f'{TEST_FOLDER}/stock_id={stock_id}')
    #print(f"reading {parquet_filename}")
    df_book_test_stock_X = pd.read_parquet(parquet_filename)
    df_book_test_stock_X['stock_id'] = stock_id
    features_var1_test = make_features_var1(df_book_test_stock_X)
    
    df_test_all_X = df_test_all.query('stock_id==@stock_id').copy()
    test_merged = pd.merge(df_test_all_X, features_var1_test, left_index=True, right_index=True, how='inner')
    test_set_predictions_X = est.predict(test_merged.reset_index()[list(features) + ['stock_id']])
    df_test_all_X['target'] = test_set_predictions_X
    df_test_predictions = pd.concat((df_test_predictions, df_test_all_X))
    
assert df_test_all.shape[0] == df_test_predictions.shape[0], "Expecting all rows to be predicted"

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 53.00it/s]

In [51] used 0.0000 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 6341.98 MiB


In [29]:
print(f"Writing {df_test_predictions.shape[0]} rows to submission.csv on {datetime.datetime.utcnow()}")
df_test_predictions.reset_index()[['row_id', 'target']].to_csv('submission.csv', index=False)

Writing 22980 rows to submission.csv on 2021-07-28 12:22:38.082873
In [29] used -52.8164 MiB RAM in 0.20s, peaked 52.82 MiB above current, total RAM usage 6646.60 MiB
